**Angie Carolina Joya  Duarte - 2322609**

**Emily Nuñez Ordoñez - 2240156**

**Hassen Ortiz Alvarez - 2177273**

Se importan las librerías necesarias y se configuran los parámetros de la librería matplotlib para que las gráficas que va a generar sean embebidas en el notebook

In [1]:
import pandas as pd
import numpy as np
import h5py
import os
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch import nn, optim
from collections import OrderedDict
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Se definen algunas variables relacionadas con las propiedades del conjunto de datos para ser utilizadas en funciones posteriores. El ancho y alto corresponden a la mitad de la resolución original de las imagenes del conjunto de datos, pues debido a limitaciones de Hardware no se pudieron procesar en su tamaño original.

In [2]:
ancho = 160
alto = 129
canales = 3
pixeles = ancho*alto*canales
cantidadClases = 5
clases = ['Banano', 'Granada', 'Kiwi', 'Mango', 'Pitaya']
batchSize = 64

Importamos el conjunto de datos de entrenamiento y de prueba, el cuál se llama Fruit Recognition y se puede encontrar en el siguiente enlace: https://www.kaggle.com/datasets/chrisfilo/fruit-recognition

El conjunto de datos se modifica para este ejercicio. Originalmente contiene 44.406 imágenes a color de 320×258 pixels, clasificadas en 25 clases. Para este proyecto se seleccionaron 5 clases: Banano, Granada, Kiwi, Mango y Pitaya; Para cada clase se seleccionaron 1500 imágenes para el entrenamiento y 500 imágenes para las pruebas, obteniendo en total 7500 imágenes de entenamiento y 2500 de prueba.

Durante la importación se define el atributo barajar (shuffle) como verdadero, esto permite que las imágenes sean cargadas en un orden aleatorio para que el modelo no tome en cuneta el orden de los datos en sus predicciones.

Además, se realiza la normalización de los valores para que estén centrados en torno a un valor promedio específico, en este caso estarían centrados alrededor de 0 y variarían en un rango aproximado de (−1,1). Esto ayuda a acelerar el entrenamiento, mejorar la convergencia y aumentar la consistencia entre los datos.

In [ ]:
transform = transforms.Compose([transforms.Resize((ancho, alto)),transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)),])
current_dir = os.getcwd()
dataset_path = os.path.join(current_dir, 'dataset', 'Frutas', 'Train')
trainset = ImageFolder(root=dataset_path, transform=transform)
trainloader = DataLoader(trainset, batch_size=batchSize, shuffle=True)

testset_path = os.path.join(current_dir, 'dataset', 'Frutas', 'Test')
testset = ImageFolder(root=testset_path, transform=transform)
testloader = DataLoader(testset, batch_size=batchSize, shuffle=True)

Se define la función view_classify, que nos permitirá graficar la probabilidad de las clases para nuestras predicciones. Esta función es una modificación de la función que se encuentra en el archivo helper.py del repositorio guía https://github.com/udacity/deep-learning-v2-pytorch/tree/master/intro-to-pytorch

In [ ]:
def view_classify(img, ps):
  ps = ps.data.numpy().squeeze()
  fig, (ax1, ax2) = plt.subplots(figsize=(16,13), ncols=2)
  ax1.imshow(img.numpy().transpose((1, 2, 0)))
  ax1.axis('off')
  ax2.barh(np.arange(cantidadClases), ps)
  ax2.set_aspect(0.1)
  ax2.set_yticks(np.arange(cantidadClases))
  ax2.set_yticklabels(clases, size='medium');
  ax2.set_title('Probabilidad de las Clases')
  ax2.set_xlim(0, 1.1)

  plt.tight_layout()

Paper: https://doi.org/10.3390/electronics12143132